In [94]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.svm import OneClassSVM
df = pd.read_csv (r'C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-6h_1.csv')

In [95]:
def transform_data(df):
    macs = ['00:0c:29:9d:9e:9e','00:80:f4:09:51:3b','48:5b:39:64:40:79','00:00:29:e6:14:0d']
    df = df[df['MAC Source'].isin(macs)]
    grouped_ip = df.groupby(df['MAC Source'])
    ip_arr_df = [grouped_ip.get_group(d) for d in df['MAC Source'].unique()]
    mac_data = []
    for df in ip_arr_df:
        #print(df['MAC Source'].unique())
        df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
        packets_per_min = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['No.'].count()
        packets_per_min = packets_per_min.fillna(0)

        time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['Delta Time'].mean()
        time_deltas = time_deltas.fillna(time_deltas.mean())
        df_final = pd.merge(packets_per_min, time_deltas, left_index=True, right_index=True)

        df_final['MAC Source'] = df['MAC Source'].unique()[0]

        mac_data.append(df_final)
        #plt.plot(packets_per_min.index, packets_per_min) 
        #print(df)
        #plt.show()
    df = pd.concat(mac_data)
    return df

def train_data_return_mod_acc(df):
    X = df[['No.', 'Delta Time']]
    y = df['MAC Source']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
    clf = DecisionTreeClassifier()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    yeet = metrics.accuracy_score(y_test, y_pred)
    return yeet, clf



In [96]:
df = df[df['MAC Source']=='00:0c:29:9d:9e:9e']
df['Source Port'] = df['Source Port'].fillna(0)
df['Destination Port'] = df['Destination Port'].fillna(0)

X = df[['Length', 'Delta Time']]
y = df['Protocol']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier()
clf = clf.fit(X,y)
y_pred = clf.predict(X)
yeet = metrics.accuracy_score(y, y_pred)
yeet




0.9989585066142063

In [97]:
# boom = r'C:\Users\carlo\Documents\College\reu_cyber\test2.csv'
# df_mitm = pd.read_csv(boom)
#df_mitm = pd.read_csv (r'C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-1h_1.csv')
df_mitm = pd.read_csv (r'C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-30m-1h_1.csv')
df_mitm = df_mitm[df_mitm['MAC Source']=='00:0c:29:9d:9e:9e']
df_mitm['Source Port'] = df_mitm['Source Port'].fillna(0)
df_mitm['Destination Port'] = df_mitm['Destination Port'].fillna(0)
X_mitm = df_mitm[['Length', 'Delta Time']]
y_mitm = df_mitm['Protocol']

y_pred_mitm = clf.predict(X_mitm)
yeet_mitm = metrics.accuracy_score(y_mitm, y_pred_mitm)
yeet_mitm

0.9905336349166457